# 2 register device to fiware on Azure AKS

change ${CORE_ROOT} to your path of `core`.

In [ ]:
export CORE_ROOT="${HOME}/core"

change ${PJ_ROOT} to your path of `example-ksce2019`.

In [ ]:
export PJ_ROOT="${HOME}/example-ksce2019"
cd ${PJ_ROOT};pwd

example)
```
/Users/user/example-ksce2019
```

## load environment variables

load from `core`

In [ ]:
source ${CORE_ROOT}/docs/environments/azure_aks/env

load from `example-ksce2019`

In [ ]:
source ${PJ_ROOT}/docs/environments/azure_aks/env

### register users to RabbitMQ

In [ ]:
for e in $(env); do
  if [[ "${e}" =~ ^MQTT__([[:alnum:]_-]+)=([[:alnum:]_-]+)$ ]]; then
    username=${BASH_REMATCH[1]}
    password=${BASH_REMATCH[2]}
    
    kubectl exec rabbitmq-0 -- rabbitmqctl add_user ${username} ${password}
    kubectl exec rabbitmq-0 -- rabbitmqctl set_permissions -p / ${username} ".*" ".*" ".*"
  fi
done

example)
```
Adding user "iotagent" ...
Error:
User "iotagent" already exists
command terminated with exit code 70
Setting permissions for user "iotagent" in vhost "/" ...
Adding user "mobile_robot" ...
Setting permissions for user "mobile_robot" in vhost "/" ...
```

In [ ]:
kubectl exec rabbitmq-0 -- rabbitmqctl list_users

example)
```
Listing users ...
user	tags
mobile_robot	[]
guest	[administrator]
iotagent	[]
```

## register mobile_robot service

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${MOBILE_ROBOT_SERVICEPATH}" -H "Content-Type: application/json" https://api.${DOMAIN}/idas/ul20/manage/iot/services/ -X POST -d @- <<__EOS__
{
  "services": [
    {
      "apikey": "${MOBILE_ROBOT_TYPE}",
      "cbroker": "http://orion:1026",
      "resource": "/iot/d",
      "entity_type": "${MOBILE_ROBOT_TYPE}"
    }
  ]
}
__EOS__

example)
```json
{}
```

### confirm registered service

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${MOBILE_ROBOT_SERVICEPATH}" https://api.${DOMAIN}/idas/ul20/manage/iot/services/ | jq .

example)
```json
{
  "count": 1,
  "services": [
    {
      "commands": [],
      "lazy": [],
      "attributes": [],
      "_id": "5d8af033359795000f6e976b",
      "resource": "/iot/d",
      "apikey": "mobile_robot",
      "service": "guide_demo",
      "subservice": "/mobile_robot",
      "__v": 0,
      "static_attributes": [],
      "internal_attributes": [],
      "entity_type": "mobile_robot"
    }
  ]
}
```

## register mobile_robot device

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${MOBILE_ROBOT_SERVICEPATH}" -H "Content-Type: application/json" https://api.${DOMAIN}/idas/ul20/manage/iot/devices/ -X POST -d @- <<__EOS__
{
  "devices": [
    {
      "device_id": "${MOBILE_ROBOT_ID}",
      "entity_name": "${MOBILE_ROBOT_ID}",
      "entity_type": "${MOBILE_ROBOT_TYPE}",
      "timezone": "Asia/Tokyo",
      "protocol": "UL20",
      "attributes": [
      ],
      "commands": [
        {
          "name": "move",
          "type": "string"
        }
      ],
      "transport": "AMQP"
    }
  ]
}
__EOS__

example)
```json
{}
```

### confirm registered device

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${MOBILE_ROBOT_SERVICEPATH}" https://api.${DOMAIN}/idas/ul20/manage/iot/devices/${MOBILE_ROBOT_ID}/ | jq .

example)
```json
{
  "device_id": "rosbot2_01",
  "service": "guide_demo",
  "service_path": "/mobile_robot",
  "entity_name": "rosbot2_01",
  "entity_type": "mobile_robot",
  "transport": "AMQP",
  "attributes": [],
  "lazy": [],
  "commands": [
    {
      "object_id": "move",
      "name": "move",
      "type": "string"
    }
  ],
  "static_attributes": [],
  "protocol": "UL20"
}
```

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${MOBILE_ROBOT_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/${MOBILE_ROBOT_ID}/ | jq .

example)
```json
{
  "id": "rosbot2_01",
  "type": "mobile_robot",
  "TimeInstant": {
    "type": "ISO8601",
    "value": " ",
    "metadata": {}
  },
  "move_info": {
    "type": "commandResult",
    "value": " ",
    "metadata": {}
  },
  "move_status": {
    "type": "commandStatus",
    "value": "UNKNOWN",
    "metadata": {}
  },
  "move": {
    "type": "string",
    "value": "",
    "metadata": {}
  }
}
```

### register `cygnus-mongo` as a subscriber of mobile_robot device

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -i -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${MOBILE_ROBOT_SERVICEPATH}" -H "Content-Type: application/json" https://api.${DOMAIN}/orion/v2/subscriptions/ -X POST -d @- <<__EOS__
{
  "subject": {
    "entities": [{
      "idPattern": "${MOBILE_ROBOT_ID}",
      "type": "${MOBILE_ROBOT_TYPE}"
    }],
    "condition": {
      "attrs": ["move_status", "move_info"]
    }
  },
  "notification": {
    "http": {
      "url": "http://cygnus-mongo:5050/notify"
    },
    "attrs": ["move_status", "move_info"],
    "attrsFormat": "legacy"
  }
}
__EOS__

example)
```
HTTP/1.1 201 Created
content-length: 0
location: /v2/subscriptions/5cd620e1d9d5db2ffb119d16
fiware-correlator: 7beb02c2-7389-11e9-88b9-46b1c9ec8ccd
date: Sat, 11 May 2019 01:09:53 GMT
x-envoy-upstream-service-time: 4
server: envoy
```

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${MOBILE_ROBOT_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/subscriptions/ | jq .

example)
```json
[
  {
    "id": "5d8af0ca0ca888199b27586a",
    "status": "active",
    "subject": {
      "entities": [
        {
          "idPattern": "rosbot2_01",
          "type": "mobile_robot"
        }
      ],
      "condition": {
        "attrs": [
          "move_status",
          "move_info"
        ]
      }
    },
    "notification": {
      "timesSent": 1,
      "lastNotification": "2019-09-25T04:44:58.00Z",
      "attrs": [
        "move_status",
        "move_info"
      ],
      "attrsFormat": "legacy",
      "http": {
        "url": "http://cygnus-mongo:5050/notify"
      },
      "lastSuccess": "2019-09-25T04:44:58.00Z",
      "lastSuccessCode": 200
    }
  }
]
```

## test mobile_robot device

### prepare a command to subscribe all topics

In [ ]:
echo "mosquitto_sub -h mqtt.${DOMAIN} -p 8883 --cafile ${CORE_ROOT}/secrets/DST_Root_CA_X3.pem -d -u iotagent -P ${MQTT__iotagent} -t /#"

### subscribe all topic

_Outside of this notebook_
1. open a ternminal.
1. run the above command displayed `prepare a command to subscribe all topics`.

### update the move attribute of mobile_robot to emulate sending the command

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -i -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${MOBILE_ROBOT_SERVICEPATH}" -H "Content-Type: application/json" https://api.${DOMAIN}/orion/v2/entities/${MOBILE_ROBOT_ID}/attrs?type=${MOBILE_ROBOT_TYPE} -X PATCH -d @-<<__EOS__
{
  "move": {
    "value": "home"
  }
}
__EOS__

example)
```
HTTP/1.1 204 No Content
content-length: 0
fiware-correlator: 84b12a5e-738d-11e9-83e5-46b1c9ec8ccd
date: Sat, 11 May 2019 01:38:46 GMT
x-envoy-upstream-service-time: 71
server: envoy
```

when executing the above command, show below messages on the opened terminal.

example)
```
Client mosq/TWXu4jyT8lK9gWpTrb received PUBLISH (d0, q0, r0, m0, '/mobile_robot/rosbot2_01/cmd', ... (20 bytes))
rosbot2_01@move|home
```

### confirm mobile_robot entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${MOBILE_ROBOT_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/${MOBILE_ROBOT_ID}/ | jq .

example)
```json
{
  "id": "rosbot2_01",
  "type": "mobile_robot",
  "TimeInstant": {
    "type": "ISO8601",
    "value": "2019-09-25T04:47:43.00Z",
    "metadata": {}
  },
  "move_info": {
    "type": "commandResult",
    "value": " ",
    "metadata": {}
  },
  "move_status": {
    "type": "commandStatus",
    "value": "PENDING",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-09-25T04:47:43.738Z"
      }
    }
  },
  "move": {
    "type": "string",
    "value": "",
    "metadata": {}
  }
}
```

### publish a MQTT message to emulate receiving the command result from mobile_robot

In [ ]:
mosquitto_pub -h mqtt.${DOMAIN} -p 8883 --cafile ${CORE_ROOT}/secrets/DST_Root_CA_X3.pem -d -u iotagent -P ${MQTT__iotagent} -t /${MOBILE_ROBOT_TYPE}/${MOBILE_ROBOT_ID}/cmdexe -m "${MOBILE_ROBOT_ID}@move|received home"

example)
```
Client mosq/zmmwCxWFQSP0u6XAFN sending CONNECT
Client mosq/zmmwCxWFQSP0u6XAFN received CONNACK (0)
Client mosq/zmmwCxWFQSP0u6XAFN sending PUBLISH (d0, q0, r0, m1, '/mobile_robot/rosbot2_01/cmdexe', ... (27 bytes))
Client mosq/zmmwCxWFQSP0u6XAFN sending DISCONNECT
```

when executing the above command, show below messages on the opened terminal.

example)
```
Client mosq/TWXu4jyT8lK9gWpTrb received PUBLISH (d0, q0, r0, m0, '/mobile_robot/rosbot2_01/cmdexe', ... (29 bytes))
rosbot2_01@move|received home
```

### confirm mobile_robot entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${MOBILE_ROBOT_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/${MOBILE_ROBOT_ID}/ | jq .

example)
```json
{
  "id": "rosbot2_01",
  "type": "mobile_robot",
  "TimeInstant": {
    "type": "ISO8601",
    "value": "2019-09-25T04:48:29.00Z",
    "metadata": {}
  },
  "move_info": {
    "type": "commandResult",
    "value": "received home",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-09-25T04:48:29.589Z"
      }
    }
  },
  "move_status": {
    "type": "commandStatus",
    "value": "OK",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-09-25T04:48:29.589Z"
      }
    }
  },
  "move": {
    "type": "string",
    "value": "",
    "metadata": {}
  }
}
```

### confirm cygnus-mongo

In [ ]:
c=$(echo sth_${MOBILE_ROBOT_SERVICEPATH}xffff${MOBILE_ROBOT_ID}xffff${MOBILE_ROBOT_TYPE} | perl -pe 's/\//x002f/g; s/\$/x0024/g; s/=/xffff/g;')
kubectl exec mongodb-0 -c mongodb-replicaset -- mongo sth_${FIWARE_SERVICE} --eval "db.getCollection(\"${c}\").find().sort({recvTime: -1})"

example)
```
MongoDB shell version v4.1.13
connecting to: mongodb://127.0.0.1:27017/sth_guide_demo?compressors=disabled&gssapiServiceName=mongodb
Implicit session: session { "id" : UUID("9a0af3f9-7d89-44e4-bc41-e87a5c800134") }
MongoDB server version: 4.1.13
{ "_id" : ObjectId("5d8af19dda9e650014954e9d"), "recvTime" : ISODate("2019-09-25T04:48:29.589Z"), "attrName" : "move_status", "attrType" : "commandStatus", "attrValue" : "OK" }
{ "_id" : ObjectId("5d8af19dda9e650014954e9e"), "recvTime" : ISODate("2019-09-25T04:48:29.589Z"), "attrName" : "move_info", "attrType" : "commandResult", "attrValue" : "received home" }
{ "_id" : ObjectId("5d8af170da9e650014954e9b"), "recvTime" : ISODate("2019-09-25T04:47:43.738Z"), "attrName" : "move_status", "attrType" : "commandStatus", "attrValue" : "PENDING" }
{ "_id" : ObjectId("5d8af0cbdbf99b00148fcc8e"), "recvTime" : ISODate("2019-09-25T04:44:58.467Z"), "attrName" : "move_status", "attrType" : "commandStatus", "attrValue" : "UNKNOWN" }
```

### confirm sth-comet

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
for a in "move_info" "move_status"; do
  curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${MOBILE_ROBOT_SERVICEPATH}" https://api.${DOMAIN}/comet/STH/v1/contextEntities/type/${MOBILE_ROBOT_TYPE}/id/${MOBILE_ROBOT_ID}/attributes/${a}?lastN=100 | jq .
done

example)
```json
{
  "contextResponses": [
    {
      "contextElement": {
        "attributes": [
          {
            "name": "move_info",
            "values": [
              {
                "recvTime": "2019-09-25T04:48:29.589Z",
                "attrType": "commandResult",
                "attrValue": "received home"
              }
            ]
          }
        ],
        "id": "rosbot2_01",
        "isPattern": false,
        "type": "mobile_robot"
      },
      "statusCode": {
        "code": "200",
        "reasonPhrase": "OK"
      }
    }
  ]
}
```
```json
{
  "contextResponses": [
    {
      "contextElement": {
        "attributes": [
          {
            "name": "move_status",
            "values": [
              {
                "recvTime": "2019-09-25T04:44:58.467Z",
                "attrType": "commandStatus",
                "attrValue": "UNKNOWN"
              },
              {
                "recvTime": "2019-09-25T04:47:43.738Z",
                "attrType": "commandStatus",
                "attrValue": "PENDING"
              },
              {
                "recvTime": "2019-09-25T04:48:29.589Z",
                "attrType": "commandStatus",
                "attrValue": "OK"
              }
            ]
          }
        ],
        "id": "rosbot2_01",
        "isPattern": false,
        "type": "mobile_robot"
      },
      "statusCode": {
        "code": "200",
        "reasonPhrase": "OK"
      }
    }
  ]
}
```